In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold

In [22]:
sample_sol = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv', index_col = 0)
sample_sol

,price
id,
1,0
2,0
3,0
4,0
5,0
...,...
4996,0
4997,0
4998,0


In [32]:
train = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv', index_col = 0)
test = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv', index_col = 0)

In [33]:
train

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712
...,...,...,...,...,...,...,...,...,...,...,...
19996,Indigo,6E-6178,Bangalore,Night,one,Early_Morning,Mumbai,Economy,7.92,45,3153
19997,AirAsia,I5-582,Kolkata,Morning,one,Afternoon,Delhi,Economy,5.83,24,3911
19998,Vistara,UK-832,Chennai,Early_Morning,two_or_more,Evening,Bangalore,Economy,35.33,17,14822


In [34]:
test

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,,
1,Air_India,AI-765,Kolkata,Evening,one,Night,Delhi,Business,28.25,2
2,Vistara,UK-747,Delhi,Early_Morning,one,Night,Mumbai,Business,13.83,34
3,Air_India,AI-570,Mumbai,Early_Morning,zero,Early_Morning,Chennai,Business,2.00,30
4,AirAsia,I5-974,Hyderabad,Night,one,Late_Night,Delhi,Economy,5.17,26
5,Air_India,AI-770,Kolkata,Night,one,Afternoon,Mumbai,Economy,16.33,35
...,...,...,...,...,...,...,...,...,...,...
4996,Air_India,AI-768,Kolkata,Afternoon,one,Morning,Bangalore,Business,17.42,15
4997,Indigo,6E-6214,Kolkata,Morning,zero,Afternoon,Mumbai,Economy,3.00,40
4998,Air_India,AI-402,Kolkata,Morning,one,Night,Mumbai,Business,11.17,37


In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 1 to 20000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  object 
 1   flight            20000 non-null  object 
 2   source_city       20000 non-null  object 
 3   departure_time    20000 non-null  object 
 4   stops             20000 non-null  object 
 5   arrival_time      20000 non-null  object 
 6   destination_city  20000 non-null  object 
 7   class             20000 non-null  object 
 8   duration          20000 non-null  float64
 9   days_left         20000 non-null  int64  
 10  price             20000 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ MB


In [36]:
train.describe()

,duration,days_left,price
count,20000.000000,20000.000000,20000.000000
mean,12.177627,25.924150,20960.281700
std,7.157944,13.624874,22775.459535
min,0.830000,1.000000,1105.000000
25%,6.830000,14.000000,4783.000000
50%,11.250000,26.000000,7425.000000
75%,16.080000,38.000000,42521.000000
max,38.580000,49.000000,114523.000000


In [37]:
train['flight'].value_counts()

flight
UK-706     208
UK-720     190
UK-822     178
UK-836     169
UK-772     169
          ... 
6E-2471      1
6E-821       1
SG-1031      1
6E-759       1
6E-7201      1
Name: count, Length: 1310, dtype: int64

In [38]:
train[['airline_code', 'flight_number']] = train['flight'].str.split('-', expand=True)
train.drop('flight', axis=1, inplace=True)
train['flight_number'] = train['flight_number'].astype(int)
train.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price,airline_code,flight_number
id,,,,,,,,,,,,
1,Vistara,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212,UK,810
2,SpiceJet,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292,SG,5094
3,Vistara,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553,UK,846
4,Vistara,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760,UK,706
5,Indigo,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712,6E,5394


In [39]:
test[['airline_code', 'flight_number']] = test['flight'].str.split('-', expand=True)
test.drop('flight', axis=1, inplace=True)
test['flight_number'] = test['flight_number'].astype(int)
test.head()

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,airline_code,flight_number
id,,,,,,,,,,,
1,Air_India,Kolkata,Evening,one,Night,Delhi,Business,28.25,2,AI,765
2,Vistara,Delhi,Early_Morning,one,Night,Mumbai,Business,13.83,34,UK,747
3,Air_India,Mumbai,Early_Morning,zero,Early_Morning,Chennai,Business,2.00,30,AI,570
4,AirAsia,Hyderabad,Night,one,Late_Night,Delhi,Economy,5.17,26,I5,974
5,Air_India,Kolkata,Night,one,Afternoon,Mumbai,Economy,16.33,35,AI,770


In [40]:
num_feat = ['duration', 'days_left', 'flight_number']
cat_feat = ['airline', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class', 'airline_code']

In [41]:
train_data = pd.get_dummies(train)
test_data = pd.get_dummies(test)
scaler = StandardScaler()

In [42]:
train_data[num_feat]=scaler.fit_transform(train_data[num_feat])
test_data[num_feat]=scaler.transform(test_data[num_feat])

features=test_data.columns.tolist()

In [ ]:
train_data

,duration,days_left,price,flight_number,airline_AirAsia,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,airline_Vistara,...,destination_city_Kolkata,destination_city_Mumbai,class_Business,class_Economy,airline_code_6E,airline_code_AI,airline_code_G8,airline_code_I5,airline_code_SG,airline_code_UK
id,,,,,,,,,,,,,,,,,,,,,
1,0.289528,-0.361418,7212,-0.307081,False,False,False,False,False,True,...,False,True,False,True,False,False,False,False,False,True
2,-1.456827,-1.388976,5292,1.867123,False,False,False,False,True,False,...,True,False,False,True,False,False,False,False,True,False
3,-0.362910,-1.535770,60553,-0.288811,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,True
4,-0.758285,0.152361,5760,-0.359863,False,False,False,False,False,True,...,False,False,False,True,False,False,False,False,False,True
5,-1.421900,-1.609167,10712,2.019378,False,False,False,True,False,False,...,False,True,False,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,-0.594826,1.400110,3153,2.417272,False,False,False,True,False,False,...,False,True,False,True,True,False,False,False,False,False
19997,-0.886817,-0.141227,3911,-0.422795,True,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
19998,3.234582,-0.655006,14822,-0.295916,False,False,False,False,False,True,...,False,False,False,True,False,False,False,False,False,True


In [44]:
def calculate_rmse(model, validation_features, validation_target):
    predicted_values = model.predict(validation_features)
    mse = mean_squared_error(validation_target, predicted_values)
    rmse = np.sqrt(mse)
    return rmse

In [46]:
rf_predictions = []
rf_rmse_scores = []

kf = KFold(n_splits=10, shuffle=True, random_state=42)

for fold_index, (train_indices, valid_indices) in enumerate(kf.split(X=train_data)):
    train_features, train_target = train_data.iloc[train_indices][features], train_data.iloc[train_indices]['price']
    valid_features, valid_target = train_data.iloc[valid_indices][features], train_data.iloc[valid_indices]['price']
    test_features = test_data[features]
    
    rf_model = RandomForestRegressor()
    rf_model.fit(train_features, train_target)
    
    rmse_score = calculate_rmse(rf_model, valid_features, valid_target)
    rf_rmse_scores.append(rmse_score)
    
    rf_predictions.append(rf_model.predict(test_features))
    
    print(f"Fold {fold_index + 1}: RMSE = {rmse_score}")

mean_rmse = np.mean(rf_rmse_scores)
print(f"\nСредний RMSE модели Random Forest: {mean_rmse}")

final_rf_predictions = np.mean(np.column_stack(rf_predictions), axis=1)

Fold 1: RMSE = 3374.483659879992
Fold 2: RMSE = 3136.4564952753512
Fold 3: RMSE = 3508.191746654204
Fold 4: RMSE = 3291.8663840625636
Fold 5: RMSE = 3006.9584002305737
Fold 6: RMSE = 2793.9444586035784
Fold 7: RMSE = 3247.221753352399
Fold 8: RMSE = 3163.2037351759145
Fold 9: RMSE = 3633.4118279718455
Fold 10: RMSE = 3385.363626316655

Средний RMSE модели Random Forest: 3254.1102087523077


In [53]:
test_predictions = rf_model.predict(test_features)
sample_sol['price'] = test_predictions
test_results = sample_sol
test_results.to_csv("test_predictions.csv")
test_results

,price
id,
1,55165.42
2,63530.46
3,23406.26
4,2457.56
5,5585.81
...,...
4996,60195.86
4997,4405.53
4998,50264.50
